# 预测分析·客户购买预测


## 练习是kesci的比赛：

本练习赛与葡萄牙银行机构的营销活动相关。这些营销活动一般以电话为基础，银行的客服人员至少联系客户一次，以确认客户是否有意愿购买该银行的产品（定期存款）。

任务是基本类型为分类任务，即预测客户是否购买该银行的产品。

## 数据指标：

 字段名称  数据类型	 字段描述
 
1 ID Int  客户唯一标识

2 age Int  客户年龄

3 job String  客户的职业

4 marital String  婚姻状况

5 education String  受教育水平

6 default String  是否有违约记录

7 balance Int  每年账户的平均余额

8 housing String  是否有住房贷款

9 loan String  是否有个人贷款

10 contact String  与客户联系的沟通方式

11 day Int  最后一次联系的时间（几号）

12 month String  最后一次联系的时间（月份）

13 duration Int  最后一次联系的交流时长

14 campaign Int  在本次活动中，与该客户交流过的次数

15 pdays Int  距离上次活动最后一次联系该客户，过去了多久（999表示没有联系过）16previousInt在本次活动之前，与该客户交流过的次数

17 poutcome String  上一次活动的结果

18 y Int  预测客户是否会订购定期存款业务

## 从sklearn 调入所需要的包

In [1]:
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np
import pandas as pd
#导入精度和召回率
from sklearn.metrics import precision_score, recall_score
import lightgbm as lgb

## 读取数据

In [2]:
path='./'
train = pd.read_csv(path+'input/train_set.csv')
test = pd.read_csv(path+'input/test_set.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         25317 non-null  int64 
 1   age        25317 non-null  int64 
 2   job        25317 non-null  object
 3   marital    25317 non-null  object
 4   education  25317 non-null  object
 5   default    25317 non-null  object
 6   balance    25317 non-null  int64 
 7   housing    25317 non-null  object
 8   loan       25317 non-null  object
 9   contact    25317 non-null  object
 10  day        25317 non-null  int64 
 11  month      25317 non-null  object
 12  duration   25317 non-null  int64 
 13  campaign   25317 non-null  int64 
 14  pdays      25317 non-null  int64 
 15  previous   25317 non-null  int64 
 16  poutcome   25317 non-null  object
 17  y          25317 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 3.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 

## 数据展示

In [3]:

train.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,12659.000000,40.935379,1357.555082,15.835289,257.732393,2.772050,40.248766,0.591737,0.116957
std,7308.532719,10.634289,2999.822811,8.319480,256.975151,3.136097,100.213541,2.568313,0.321375
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6330.000000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,12659.000000,39.000000,448.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,18988.000000,48.000000,1435.000000,21.000000,317.000000,3.000000,-1.000000,0.000000,0.000000
max,25317.000000,95.000000,102127.000000,31.000000,3881.000000,55.000000,854.000000,275.000000,1.000000


In [4]:
test.describe()

,ID,age,balance,day,duration,campaign,pdays,previous
count,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000
mean,30743.500000,41.040638,1393.157298,15.680151,257.206137,2.770365,39.954755,0.546443
std,3132.846895,10.652369,3318.497101,8.302317,250.480906,3.063481,99.524056,1.805938
min,25318.000000,18.000000,-2604.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,28030.750000,33.000000,72.000000,8.000000,102.000000,1.000000,-1.000000,0.000000
50%,30743.500000,39.000000,450.000000,16.000000,181.000000,2.000000,-1.000000,0.000000
75%,33456.250000,49.000000,1440.250000,21.000000,322.000000,3.000000,-1.000000,0.000000
max,36169.000000,94.000000,81204.000000,31.000000,3102.000000,58.000000,871.000000,51.000000


In [5]:
train.corr()

,ID,age,balance,day,duration,campaign,pdays,previous,y
ID,1.000000,0.008465,0.032719,-0.020171,0.229149,-0.038265,0.065807,0.047028,0.556627
age,0.008465,1.000000,0.093740,-0.016070,0.000416,0.006171,-0.026431,0.006575,0.029916
balance,0.032719,0.093740,1.000000,0.010245,0.026042,-0.010419,0.001032,0.015792,0.057564
day,-0.020171,-0.016070,0.010245,1.000000,-0.031946,0.168830,-0.092892,-0.050706,-0.031886
duration,0.229149,0.000416,0.026042,-0.031946,1.000000,-0.087780,0.000040,0.001315,0.394746
campaign,-0.038265,0.006171,-0.010419,0.168830,-0.087780,1.000000,-0.089224,-0.031667,-0.075173
pdays,0.065807,-0.026431,0.001032,-0.092892,0.000040,-0.089224,1.000000,0.411688,0.107565
previous,0.047028,0.006575,0.015792,-0.050706,0.001315,-0.031667,0.411688,1.000000,0.088337
y,0.556627,0.029916,0.057564,-0.031886,0.394746,-0.075173,0.107565,0.088337,1.000000


In [5]:
train.job.value_counts()

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

In [6]:
train.education.value_counts()

secondary    12957
tertiary      7447
primary       3848
unknown       1065
Name: education, dtype: int64

### train 与 test连接之前，将test的预测值为-1，方便取出test。

In [30]:

test['y'] = -1
print(len(test.columns))

18


In [8]:
data=pd.concat([train,test])

In [9]:
data.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,36169.00000,36169.000000,36169.000000,36169.000000,36169.000000,36169.000000,36169.000000,36169.000000,36169.000000
mean,18085.00000,40.966961,1368.237026,15.788742,257.574497,2.771545,40.160552,0.578147,-0.218170
std,10441.23528,10.639679,3098.876172,8.314523,255.040668,3.114445,100.005886,2.365582,0.578198
min,1.00000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,-1.000000
25%,9043.00000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,-1.000000
50%,18085.00000,39.000000,449.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,27127.00000,48.000000,1437.000000,21.000000,318.000000,3.000000,-1.000000,0.000000,0.000000
max,36169.00000,95.000000,102127.000000,31.000000,3881.000000,58.000000,871.000000,275.000000,1.000000


In [9]:
data = train.append(test).reset_index(drop=True)
# data = pd.concat([train,test]).reset_index(drop=True)
print(data.head())

   ID  age         job   marital  education default  balance housing loan  \
0   1   43  management   married   tertiary      no      291     yes   no   
1   2   42  technician  divorced    primary      no     5076     yes   no   
2   3   47      admin.   married  secondary      no      104     yes  yes   
3   4   28  management    single  secondary      no     -994     yes  yes   
4   5   42  technician  divorced  secondary      no     2974     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0   unknown    9   may       150         2     -1         0  unknown  0  
1  cellular    7   apr        99         1    251         2    other  0  
2  cellular   14   jul        77         2     -1         0  unknown  0  
3  cellular   18   jul       174         2     -1         0  unknown  0  
4   unknown   21   may       187         5     -1         0  unknown  0  


In [11]:
os.system('pip install numba')

0

In [13]:
path='./'
import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
# import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection 
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
    #warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 

In [15]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
# for i in tqdm_notebook(cat_col):
#     lbl = LabelEncoder()
# #     data['count_' + i] = data.groupby([i])[i].transform('count')
#     data[i] = lbl.fit_transform(data[i].astype(str))
    
for i in cat_col:
    data[i] = data[i].astype('category')

In [16]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


1ID Int客户唯一标识

2age Int客户年龄

3job String客户的职业

4marital String婚姻状况

5education String受教育水平

6default String是否有违约记录

7balance Int每年账户的平均余额

8housing String是否有住房贷款

9loan String是否有个人贷款

10contact String与客户联系的沟通方式

11day Int最后一次联系的时间（几号）

12month String最后一次联系的时间（月份）

13duration Int最后一次联系的交流时长

14campaign Int在本次活动中，与该客户交流过的次数

15pdays Int距离上次活动最后一次联系该客户，过去了多久（999表示没有联系过）16previousInt在本次活动之前，与该客户交流过的次数

17poutcome String上一次活动的结果

18y Int预测客户是否会订购定期存款业务

In [13]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [14]:
feats = [i for i in data.columns if i not in ['ID','y']]
feats

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome']

In [17]:
print(len(feats))

16


## baseline 1：字符串的类型还是字符串

In [18]:
import lightgbm as lgb
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )
data1 =data[data['y']!=-1][feats]
label1 =data[data['y']!=-1]['y']
testx= data[data['y']==-1][feats]

train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
model.fit(train_x,train_y)
test_pre = model.predict_proba(testx)[:,1]#, num_iteration=model.best_iteration

In [17]:
model.predict(testx)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [29]:
test_pre[20]

0.013868511704910082

In [18]:
print(len(test_pre))

10852


In [32]:
pre = data[data['y']== -1][['ID']]
pre['pred'] = test_pre
print(len(pre))

10852


In [33]:
pre.head()

,ID,pred
0,25318,0.026553
1,25319,0.003254
2,25320,0.000630
3,25321,0.931781
4,25322,0.026751


In [34]:
pre.to_csv('baseline1.csv',index=False )

## baseline 2 :  字符串的特征 + 5折交叉验证

In [22]:

from sklearn.model_selection import KFold,StratifiedKFold
n_splits=5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)


train_x = data[data['y']!=-1][feats]
train_y = data[data['y']!=-1]['y']


res=data[data['y']==-1][['ID']]
res['pred'] = 0


test_x= data[data['y']==-1][feats]

for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    model.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],categorical_feature=cat_col,eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/n_splits
res.to_csv('./submission5_fold.csv', index=False)

[1]	training's auc: 0.774481	valid_1's auc: 0.794136
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.917336	valid_1's auc: 0.910339
[3]	training's auc: 0.925961	valid_1's auc: 0.916028
[4]	training's auc: 0.928113	valid_1's auc: 0.919957
[5]	training's auc: 0.928141	valid_1's auc: 0.921038
[6]	training's auc: 0.927656	valid_1's auc: 0.921069
[7]	training's auc: 0.929385	valid_1's auc: 0.923216
[8]	training's auc: 0.930309	valid_1's auc: 0.922831
[9]	training's auc: 0.931673	valid_1's auc: 0.925088
[10]	training's auc: 0.932251	valid_1's auc: 0.925163
[11]	training's auc: 0.932738	valid_1's auc: 0.924581
[12]	training's auc: 0.933105	valid_1's auc: 0.924719
[13]	training's auc: 0.933735	valid_1's auc: 0.925742
[14]	training's auc: 0.93446	valid_1's auc: 0.926272
[15]	training's auc: 0.935027	valid_1's auc: 0.926838
[16]	training's auc: 0.935055	valid_1's auc: 0.926867
[17]	training's auc: 0.935443	valid_1's auc: 0.927054
[18]	training's auc: 0.935891

[300]	training's auc: 0.964093	valid_1's auc: 0.936112
[301]	training's auc: 0.964163	valid_1's auc: 0.936133
[302]	training's auc: 0.964239	valid_1's auc: 0.936098
[303]	training's auc: 0.964344	valid_1's auc: 0.936101
[304]	training's auc: 0.964408	valid_1's auc: 0.93606
[305]	training's auc: 0.964477	valid_1's auc: 0.936098
[306]	training's auc: 0.964534	valid_1's auc: 0.936063
[307]	training's auc: 0.964613	valid_1's auc: 0.936102
[308]	training's auc: 0.96467	valid_1's auc: 0.936101
[309]	training's auc: 0.964758	valid_1's auc: 0.936122
[310]	training's auc: 0.964808	valid_1's auc: 0.936169
[311]	training's auc: 0.964866	valid_1's auc: 0.936148
[312]	training's auc: 0.964929	valid_1's auc: 0.936137
[313]	training's auc: 0.965008	valid_1's auc: 0.936182
[314]	training's auc: 0.965047	valid_1's auc: 0.936201
[315]	training's auc: 0.965141	valid_1's auc: 0.936188
[316]	training's auc: 0.965199	valid_1's auc: 0.936201
[317]	training's auc: 0.96524	valid_1's auc: 0.936192
[318]	trainin

[61]	training's auc: 0.942234	valid_1's auc: 0.92746
[62]	training's auc: 0.942306	valid_1's auc: 0.927359
[63]	training's auc: 0.942391	valid_1's auc: 0.927432
[64]	training's auc: 0.942451	valid_1's auc: 0.927445
[65]	training's auc: 0.942472	valid_1's auc: 0.927411
[66]	training's auc: 0.942562	valid_1's auc: 0.927543
[67]	training's auc: 0.942557	valid_1's auc: 0.927567
[68]	training's auc: 0.94271	valid_1's auc: 0.927669
[69]	training's auc: 0.942747	valid_1's auc: 0.927704
[70]	training's auc: 0.942736	valid_1's auc: 0.927661
[71]	training's auc: 0.942795	valid_1's auc: 0.927626
[72]	training's auc: 0.942869	valid_1's auc: 0.927594
[73]	training's auc: 0.943163	valid_1's auc: 0.927706
[74]	training's auc: 0.943149	valid_1's auc: 0.927727
[75]	training's auc: 0.943206	valid_1's auc: 0.927745
[76]	training's auc: 0.943284	valid_1's auc: 0.92778
[77]	training's auc: 0.943355	valid_1's auc: 0.927864
[78]	training's auc: 0.943496	valid_1's auc: 0.928005
[79]	training's auc: 0.943984	v

[360]	training's auc: 0.967986	valid_1's auc: 0.93485
[361]	training's auc: 0.968042	valid_1's auc: 0.934855
[362]	training's auc: 0.968083	valid_1's auc: 0.934842
[363]	training's auc: 0.968142	valid_1's auc: 0.934804
[364]	training's auc: 0.968192	valid_1's auc: 0.934823
[365]	training's auc: 0.968262	valid_1's auc: 0.934842
[366]	training's auc: 0.968308	valid_1's auc: 0.934832
[367]	training's auc: 0.968362	valid_1's auc: 0.934831
[368]	training's auc: 0.968397	valid_1's auc: 0.934863
[369]	training's auc: 0.968459	valid_1's auc: 0.934909
[370]	training's auc: 0.968507	valid_1's auc: 0.934915
[371]	training's auc: 0.968594	valid_1's auc: 0.934919
[372]	training's auc: 0.968645	valid_1's auc: 0.934926
[373]	training's auc: 0.968712	valid_1's auc: 0.934956
[374]	training's auc: 0.968776	valid_1's auc: 0.934967
[375]	training's auc: 0.968836	valid_1's auc: 0.934945
[376]	training's auc: 0.968909	valid_1's auc: 0.934952
[377]	training's auc: 0.968966	valid_1's auc: 0.934959
[378]	train

[105]	training's auc: 0.946168	valid_1's auc: 0.939283
[106]	training's auc: 0.946238	valid_1's auc: 0.939285
[107]	training's auc: 0.946478	valid_1's auc: 0.939438
[108]	training's auc: 0.946519	valid_1's auc: 0.939446
[109]	training's auc: 0.946736	valid_1's auc: 0.939543
[110]	training's auc: 0.946877	valid_1's auc: 0.939566
[111]	training's auc: 0.946973	valid_1's auc: 0.93956
[112]	training's auc: 0.947029	valid_1's auc: 0.939617
[113]	training's auc: 0.947089	valid_1's auc: 0.939595
[114]	training's auc: 0.947155	valid_1's auc: 0.939607
[115]	training's auc: 0.947377	valid_1's auc: 0.939849
[116]	training's auc: 0.94744	valid_1's auc: 0.939867
[117]	training's auc: 0.947513	valid_1's auc: 0.939891
[118]	training's auc: 0.947574	valid_1's auc: 0.939942
[119]	training's auc: 0.947621	valid_1's auc: 0.939892
[120]	training's auc: 0.947745	valid_1's auc: 0.939863
[121]	training's auc: 0.947794	valid_1's auc: 0.939875
[122]	training's auc: 0.947871	valid_1's auc: 0.939949
[123]	traini

[26]	training's auc: 0.93735	valid_1's auc: 0.929297
[27]	training's auc: 0.937499	valid_1's auc: 0.929254
[28]	training's auc: 0.937717	valid_1's auc: 0.929707
[29]	training's auc: 0.937657	valid_1's auc: 0.929476
[30]	training's auc: 0.937694	valid_1's auc: 0.929502
[31]	training's auc: 0.937753	valid_1's auc: 0.929208
[32]	training's auc: 0.937658	valid_1's auc: 0.928947
[33]	training's auc: 0.937962	valid_1's auc: 0.929199
[34]	training's auc: 0.938108	valid_1's auc: 0.929064
[35]	training's auc: 0.938283	valid_1's auc: 0.929477
[36]	training's auc: 0.938422	valid_1's auc: 0.929779
[37]	training's auc: 0.938424	valid_1's auc: 0.929648
[38]	training's auc: 0.938688	valid_1's auc: 0.929817
[39]	training's auc: 0.938939	valid_1's auc: 0.929982
[40]	training's auc: 0.938913	valid_1's auc: 0.929751
[41]	training's auc: 0.93934	valid_1's auc: 0.930352
[42]	training's auc: 0.939341	valid_1's auc: 0.930435
[43]	training's auc: 0.93932	valid_1's auc: 0.930439
[44]	training's auc: 0.939806	v

[325]	training's auc: 0.964907	valid_1's auc: 0.940286
[326]	training's auc: 0.96496	valid_1's auc: 0.940275
[327]	training's auc: 0.965027	valid_1's auc: 0.940254
[328]	training's auc: 0.965108	valid_1's auc: 0.940303
[329]	training's auc: 0.965186	valid_1's auc: 0.940298
[330]	training's auc: 0.965271	valid_1's auc: 0.940316
[331]	training's auc: 0.965367	valid_1's auc: 0.940374
[332]	training's auc: 0.965415	valid_1's auc: 0.940365
[333]	training's auc: 0.965476	valid_1's auc: 0.940356
[334]	training's auc: 0.965528	valid_1's auc: 0.940354
[335]	training's auc: 0.965574	valid_1's auc: 0.940339
[336]	training's auc: 0.965652	valid_1's auc: 0.940338
[337]	training's auc: 0.965714	valid_1's auc: 0.940313
[338]	training's auc: 0.965788	valid_1's auc: 0.940323
[339]	training's auc: 0.965854	valid_1's auc: 0.94033
[340]	training's auc: 0.965894	valid_1's auc: 0.940324
[341]	training's auc: 0.965933	valid_1's auc: 0.940325
[342]	training's auc: 0.966029	valid_1's auc: 0.940339
[343]	traini

[623]	training's auc: 0.979513	valid_1's auc: 0.940952
[624]	training's auc: 0.979535	valid_1's auc: 0.940962
[625]	training's auc: 0.979564	valid_1's auc: 0.940959
[626]	training's auc: 0.97961	valid_1's auc: 0.940957
[627]	training's auc: 0.979627	valid_1's auc: 0.940964
[628]	training's auc: 0.979664	valid_1's auc: 0.940978
[629]	training's auc: 0.979705	valid_1's auc: 0.940971
[630]	training's auc: 0.979737	valid_1's auc: 0.940958
[631]	training's auc: 0.979781	valid_1's auc: 0.940966
[632]	training's auc: 0.979818	valid_1's auc: 0.940918
[633]	training's auc: 0.979858	valid_1's auc: 0.940948
[634]	training's auc: 0.979892	valid_1's auc: 0.940948
[635]	training's auc: 0.979923	valid_1's auc: 0.940983
[636]	training's auc: 0.979961	valid_1's auc: 0.940971
[637]	training's auc: 0.979984	valid_1's auc: 0.940964
[638]	training's auc: 0.980008	valid_1's auc: 0.940961
[639]	training's auc: 0.980035	valid_1's auc: 0.940985
[640]	training's auc: 0.980063	valid_1's auc: 0.940988
[641]	train

[180]	training's auc: 0.956136	valid_1's auc: 0.928233
[181]	training's auc: 0.95617	valid_1's auc: 0.928201
[182]	training's auc: 0.956229	valid_1's auc: 0.928107
[183]	training's auc: 0.956322	valid_1's auc: 0.928186
[184]	training's auc: 0.956365	valid_1's auc: 0.928203
[185]	training's auc: 0.95645	valid_1's auc: 0.92818
[186]	training's auc: 0.956514	valid_1's auc: 0.928218
[187]	training's auc: 0.956577	valid_1's auc: 0.928187
[188]	training's auc: 0.956655	valid_1's auc: 0.928231
[189]	training's auc: 0.956732	valid_1's auc: 0.928188
[190]	training's auc: 0.95677	valid_1's auc: 0.928165
[191]	training's auc: 0.956867	valid_1's auc: 0.928198
[192]	training's auc: 0.956931	valid_1's auc: 0.928169
[193]	training's auc: 0.957049	valid_1's auc: 0.928264
[194]	training's auc: 0.957136	valid_1's auc: 0.928276
[195]	training's auc: 0.957235	valid_1's auc: 0.928265
[196]	training's auc: 0.95733	valid_1's auc: 0.928297
[197]	training's auc: 0.957421	valid_1's auc: 0.928323
[198]	training'

[478]	training's auc: 0.974462	valid_1's auc: 0.930206
[479]	training's auc: 0.97449	valid_1's auc: 0.930211
[480]	training's auc: 0.974518	valid_1's auc: 0.930231
[481]	training's auc: 0.974575	valid_1's auc: 0.930212
[482]	training's auc: 0.97463	valid_1's auc: 0.930222
[483]	training's auc: 0.974664	valid_1's auc: 0.930215
[484]	training's auc: 0.974701	valid_1's auc: 0.930226
[485]	training's auc: 0.974732	valid_1's auc: 0.930232
[486]	training's auc: 0.974768	valid_1's auc: 0.930218
[487]	training's auc: 0.974804	valid_1's auc: 0.930225
[488]	training's auc: 0.974854	valid_1's auc: 0.930242
[489]	training's auc: 0.9749	valid_1's auc: 0.930254
[490]	training's auc: 0.974933	valid_1's auc: 0.930238
[491]	training's auc: 0.974973	valid_1's auc: 0.930235
[492]	training's auc: 0.975026	valid_1's auc: 0.930204
[493]	training's auc: 0.975075	valid_1's auc: 0.930215
[494]	training's auc: 0.97512	valid_1's auc: 0.930216
[495]	training's auc: 0.975163	valid_1's auc: 0.930219
[496]	training'

In [23]:
train_y.shape

(25317,)

## 请思考，将字符串的特征变换数字类型的特征，使用LabelEncoder() ,做出一个baseline3的结果，以及在baseline3的基础上加一个5折交叉验证